In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:

import bz2
from collections import Counter
import re
import nltk
import numpy as np
nltk.download('punkt')

train_file = bz2.BZ2File('/content/drive/My Drive/train.ft.txt.bz2')
test_file = bz2.BZ2File('/content/drive/My Drive/test.ft.txt.bz2')

train_file = train_file.readlines()
test_file = test_file.readlines()

In [0]:
print(train_file[67])
print(len(train_file))


In [0]:
num_train = 800000  # We're training on the first 800,000 reviews in the dataset
num_test = 200000  # Using 200,000 reviews from test set
# num_train = len(train_file)
# num_test = len(test_file)
train_file = [x.decode('utf-8') for x in train_file[:num_train]]
test_file = [x.decode('utf-8') for x in test_file[:num_test]]


In [0]:
print(train_file[67])
print(len(train_file))

In [0]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file]
train_sentences = [x.split(' ', 1)[1][:-1].lower() for x in train_file]

test_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test_file]
test_sentences = [x.split(' ', 1)[1][:-1].lower() for x in test_file]


In [0]:
print(len(train_labels))
print(train_labels[67])
print(train_sentences[67])


In [0]:
# Some simple cleaning of data
for i in range(len(train_sentences)):
    train_sentences[i] = re.sub('\d','0',train_sentences[i])

for i in range(len(test_sentences)):
    test_sentences[i] = re.sub('\d','0',test_sentences[i])


In [0]:
print(train_sentences[67])


In [0]:
# Modify URLs to <url>
for i in range(len(train_sentences)):
    if 'www.' in train_sentences[i] or 'http:' in train_sentences[i] or 'https:' in train_sentences[i] or '.com' in train_sentences[i]:
        train_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", train_sentences[i])
        
for i in range(len(test_sentences)):
    if 'www.' in test_sentences[i] or 'http:' in test_sentences[i] or 'https:' in test_sentences[i] or '.com' in test_sentences[i]:
        test_sentences[i] = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", "<url>", test_sentences[i])


In [0]:
with_url = 0
for ii, s in enumerate(train_sentences):
    if '<url>' in s:
        with_url = ii
        print(ii)
        break
print(train_file[5])
print(train_sentences[with_url])


In [0]:
words = Counter()  # Dictionary that will map a word to the number of times it appeared in all the training sentences
for i, sentence in enumerate(train_sentences):
    # The sentences will be stored as a list of words/tokens
    train_sentences[i] = []
    for word in nltk.word_tokenize(sentence):  # Tokenizing the words
        words.update([word.lower()])  # Converting all the words to lowercase
        train_sentences[i].append(word)
    if i%20000 == 0:
        print(str((i*100)/num_train) + "% done")
print("100% done")


In [0]:
print(train_sentences[67])


In [0]:
words = {k:v for k,v in words.items() if v>1}
# Sorting the words according to the number of appearances, with the most common word being first
words = sorted(words, key=words.get, reverse=True)


In [0]:
print(len(words))
print(words[100])


In [0]:
words = ['_PAD','_UNK'] + words
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}


In [0]:
print(word2idx['the'])
print(idx2word[100])


In [0]:
for i, sentence in enumerate(train_sentences):
    # Looking up the mapping dictionary and assigning the index to the respective words
    train_sentences[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]

for i, sentence in enumerate(test_sentences):
    # For test sentences, we have to tokenize the sentences as well
    test_sentences[i] = [word2idx[word.lower()] if word.lower() in word2idx else 0 for word in nltk.word_tokenize(sentence)]


In [0]:
print(len(train_sentences[67]))
print(train_sentences[67])


In [0]:
more_than_200 = 0
for ii, s in enumerate(train_sentences):
    if len(s) > 200:
        more_than_200 = ii
        print(len(s))
        print(ii)
        break
print(train_file[more_than_200])
print(train_sentences[more_than_200])


In [0]:
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

seq_len = 200  # The length that the sentences will be padded/shortened to

train_sentences = pad_input(train_sentences, seq_len)
test_sentences = pad_input(test_sentences, seq_len)

# Converting our labels into numpy arrays
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


In [0]:
print(len(train_sentences[67]))
print(train_sentences[67])


In [0]:

print(len(train_sentences[more_than_200]))
print(train_sentences[more_than_200])

In [0]:
split_frac = 0.5 # 50% validation, 50% test
split_id = int(split_frac * len(test_sentences))
val_sentences, test_sentences = test_sentences[:split_id], test_sentences[split_id:]
val_labels, test_labels = test_labels[:split_id], test_labels[split_id:]

In [0]:
print(len(val_sentences))


In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

train_data = TensorDataset(torch.from_numpy(train_sentences), torch.from_numpy(train_labels))
val_data = TensorDataset(torch.from_numpy(val_sentences), torch.from_numpy(val_labels))
test_data = TensorDataset(torch.from_numpy(test_sentences), torch.from_numpy(test_labels))

batch_size = 400

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


In [0]:
class SentimentNet(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        super(SentimentNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size = x.size(0)
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [0]:
vocab_size = len(word2idx) + 1
output_size = 1
embedding_dim = 400
hidden_dim = 512
n_layers = 2

model = SentimentNet(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)
model.to(device)

lr=0.005
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


In [0]:
hh = model.init_hidden(batch_size)
hh = tuple([e.data for e in hh])
print(hh[1].shape)
print(hh[1])


In [0]:
inputs = next(iter(train_loader)) # 取一段training data出來
x = inputs[0].to(device) #把資料移入gpu
batch_num = x.size(0) #取得批次大小，256
x = x.long() # 把輸入值轉成長整數
print(x.shape) # 看看x的型狀長什麼樣子
print(x) #印出一個x來看看


In [0]:
embedding = nn.Embedding(vocab_size, embedding_dim) # 把x從one-hot的詞彙大小轉成word2vec的大小
embedding = embedding.to(device) #將這個函數放入gpu


In [0]:
embeds = embedding(x) # 將詞彙轉成wordvec
embeds.shape #看看型狀


In [0]:
lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, dropout=0.5, batch_first=True) # 建立一個lstm
lstm = lstm.to(device) #將lstm移入gpu
lstm_out, hidden = lstm(embeds, hh) #讓剛才的資料跑過lstm，產生輸出值及新的hidden值


In [0]:
print(lstm_out.shape) #看看從lstm跑出來的東西型狀
print(hidden[0].shape) # 看看hidden第一個值的形狀


In [0]:
lstm_out = lstm_out.contiguous().view(-1, hidden_dim) #把lstm的輸出值攤平
lstm_out.shape #看看形狀


In [0]:
dropout = nn.Dropout(0.5) #設定dropout
out = dropout(lstm_out) # 讓輸出值走一次dropout


In [0]:
print(out.shape) # 查看輸出值的形狀

In [0]:
fc = nn.Linear(hidden_dim, 1) #定義最後一層fc
fc = fc.to(device) #移入gpu
out = fc(out) #讓輸入出值走一次fc
print(out.shape) # 看一下形狀


In [0]:
sigmoid = nn.Sigmoid() # 讓輸出值介於0和1之間
sigmoid = sigmoid.to(device) #將sigmoid移入gpu


In [0]:
out = sigmoid(out) #讓輸出值走一次sigmoid
print(out.shape) #看看形狀


In [0]:
out = out.view(batch_size, -1) #把輸出值還原成(批次大小，句子長度)
print(out.shape) # 看形狀


In [0]:
out = out[:,-1] #取出最後結果值
print(out.shape)# 看形狀


In [0]:
print(out) # 看每批256個輸出值


In [0]:
epochs = 2
counter = 0
print_every = 1000
clip = 5
valid_loss_min = np.Inf

model.train()
for i in range(epochs):
    h = model.init_hidden(batch_size)
    
    for inputs, labels in train_loader:
        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if counter%print_every == 0:
            val_h = model.init_hidden(batch_size)
            val_losses = []
            model.eval()
            for inp, lab in val_loader:
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
                
            model.train()
            print("Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './state_dict.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)


In [0]:
# Loading the best model
model.load_state_dict(torch.load('./state_dict.pt'))

test_losses = []
num_correct = 0
h = model.init_hidden(batch_size)

model.eval()
for inputs, labels in test_loader:
    h = tuple([each.data for each in h])
    inputs, labels = inputs.to(device), labels.to(device)
    output, h = model(inputs, h)
    test_loss = criterion(output.squeeze(), labels.float())
    test_losses.append(test_loss.item())
    pred = torch.round(output.squeeze())  # Rounds the output to 0/1
    correct_tensor = pred.eq(labels.float().view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}%".format(test_acc*100))
